# Practica once

Grupo 14:
* Joaquín Ibáñez Penalva
* Aurora Zuoris

Para la realización de esta práctica se usará la librería de numpy, pandas, matplotlib, y sklearn.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('wisconsin diagnostic breast cancer.csv')
X = data.iloc[:, 2:].values
y = data.iloc[:, 1].values

X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

## Ejercicio uno

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# Crear el modelo de árbol de decisión
model = DecisionTreeClassifier(random_state=42)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Predecir la clase de los datos de test
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
acc = accuracy_score(y_test, y_pred)
acc2 = cross_val_score(model, X, y, cv=10).mean()
print(f"Accuracy del modelo por defecto: {acc:.3f}")
print(f"Accuracy del modelo con validación cruzada: {acc2:.3f}")

La validación cruzada lo que hace es hacer la división de datos en train y test varias veces, y calcular la media de los resultados. Esto es útil para evitar que el resultado dependa de la partición de los datos.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score

# Definir los valores de los parámetros a explorar
param_grid = {
    'max_depth': range(1, 10),
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
}

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid=param_grid, scoring='recall', cv=10)
y_train_bool = y_train == 'M'
# Entrenar el objeto GridSearchCV
grid_search.fit(X_train, y_train_bool)


# Obtener los mejores parámetros y la accuracy correspondiente
best_params = grid_search.best_params_
best_acc = grid_search.best_score_

print(f"Mejores parámetros: {best_params}")
print(f"Accuracy con los mejores parámetros: {best_acc:.3f}")

Como se puede comprobar, fijandose en poner los mejores parámetros el accuracy es de 0.96% practicamente, mientras que con un árbol por defecto es de 0.93% usando validación cruzada.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
tree = DecisionTreeClassifier(**best_params, random_state=42)
tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=tree.classes_)
disp.plot()

Al ser un tema medico, nos hemos fijado en el mejor arbol teniendo en cuenta que tenga la menor tasa de falsos negativos posible, ya que es peor que un paciente tenga cancer y no se le diagnostique, que un paciente que no tenga cancer y se le diagnostique. Esto queda demostrado con la matriz de confusión, que da 0 en falsos negativos.

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

# Mostrar gráficamente los árboles obtenidos con los parámetros por defecto y los mejores hiperparámetros
plt.figure(figsize=(15,15))
plt.subplot(2,1,1)
plot_tree(tree, filled=True)
plt.title("Árbol de decisión con mejores hiperparámetros")
plt.subplot(2,1,2)
plot_tree(model, filled=True)
plt.title("Árbol de decisión con parámetros por defecto")
plt.show()

Además de ser mejor el arbol con los mejores parámetros, es más sencillo de interpretar, ya que tiene menos nodos y menor profundidad.

## Ejercicio dos